In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

#InceptionV3模型
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)  

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

87910968/87910968 [==============================] - 35s 0us/step


In [8]:
import os
import shutil
import numpy as np

dataset_dir = 'D:/tester01'
train_ratio = 0.4
val_ratio = 0.3
test_ratio = 0.3

classes = ['cataract', 'normal', 'surgery']

counts = {
    'train': {cls: 0 for cls in classes},
    'val': {cls: 0 for cls in classes},
    'test': {cls: 0 for cls in classes}
}

for set_type in ['train', 'val', 'test']:
    for cls in classes:
        os.makedirs(os.path.join(dataset_dir, set_type, cls), exist_ok=True)

for cls in classes:
    src_dir = os.path.join(dataset_dir, cls)
    images = os.listdir(src_dir)
    
    np.random.shuffle(images)

    train_split = int(len(images) * train_ratio)
    val_split = train_split + int(len(images) * val_ratio)

    for i, img in enumerate(images):
        if i < train_split:
            dst_dir = os.path.join(dataset_dir, 'train', cls)
            counts['train'][cls] += 1
        elif i < val_split:
            dst_dir = os.path.join(dataset_dir, 'val', cls)
            counts['val'][cls] += 1
        else:
            dst_dir = os.path.join(dataset_dir, 'test', cls)
            counts['test'][cls] += 1
        
        shutil.copy2(os.path.join(src_dir, img), os.path.join(dst_dir, img))

for set_type in ['train', 'val', 'test']:
    print(f"\n{set_type.capitalize()} set:")
    total = 0
    for cls in classes:
        num_images = counts[set_type][cls]
        print(f"  {cls}: {num_images} images")
        total += num_images
    print(f"  Total: {total} images")

print("\n数据集划分完成。")


Train set:
  cataract: 457 images
  normal: 310 images
  surgery: 457 images
  Total: 1224 images

Val set:
  cataract: 343 images
  normal: 232 images
  surgery: 343 images
  Total: 918 images

Test set:
  cataract: 344 images
  normal: 234 images
  surgery: 344 images
  Total: 922 images

数据集划分完成。


In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical')


model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy}, Test loss: {test_loss}')

In [13]:
model.save('cataract.h5')